In [32]:
import numpy as np
import pandas as pd
import os

In [33]:
def find_project_root(marker=".git"):
    """
    从当前目录向上搜索，直到找到包含指定标记目录的父目录，将其视为项目根目录。

    Args:
        marker: 用于标识项目根目录的子目录名称列表，默认为 ['.git', '.idea']。
    Returns:
        str: 项目根目录的绝对路径，如果未找到标记目录，则返回 None。
    """
    # 获取当前脚本或 Notebook 的工作目录
    # 在 Jupyter Notebook 中，os.getcwd() 通常是 Notebook 文件所在的目录
    current_dir = os.path.abspath(os.getcwd())
    while True:

        # 检查当前目录是否包含任何一个标记目录
        if os.path.exists(os.path.join(current_dir, marker)):
            return current_dir  # 找到根目录，返回

        # 向上移动到父目录
        parent_dir = os.path.dirname(current_dir)
        current_dir = parent_dir


def clean_food_code(code):
    """
    标准化食物编码：。
    1. 将编码转换为字符串并去除首尾空白。
    2. 分离出可能的末尾 'x' 和前面的主体部分。
    3. 如果是纯数字，根据长度（<6) 决定是否补零。
    4. 将处理好的数字部分与末尾的 'x' 重新组合。
    5. 如果主体部分不全由数字组成，返回原始字符串。
    """
    # 1. 将编码转换为字符串并去除首尾空白
    code_str = str(code).strip()  # 2. 分离出可能的末尾 'x' 和前面的主体部分
    numeric_part = code_str
    suffix_x = ""
    # 检查是否以 'x' 结尾（不区分大小写）
    if code_str.lower().endswith("x"):
        numeric_part = code_str[:-1]  # 取除最后一个字符外的所有部分作为数字主体
        suffix_x = code_str[-1]  # 获取最后一个字符，即 'x' 或 'X'

    # 3. 根据长度决定是否补零
    if len(numeric_part) < 6:
        # 长度小于6，补零至6位
        padded_numeric_part = numeric_part.zfill(6)
        # 6. 将处理好的数字部分与末尾的 'x' 重新组合
        return padded_numeric_part + suffix_x
    else:
        # 长度等于或大于6（且假设没有 >6 情况，或大于等于6都不补零）
        # 直接返回原字符串，因为数字主体已经是6位或更长，且末尾的 x 也已包含在原字符串中
        return code_str

## 清理食物编码

In [34]:
project_root_path = find_project_root()

file_names = ['附件1.xlsx', '附件2.xlsx', '附件3.xlsx']
data_subdirectory = '处理后的数据'
file_path = [os.path.join(project_root_path, data_subdirectory, name) for name in file_names]


In [35]:
def clean_food_code_main(df):
    df['食物编码'] = df['食物编码'].apply(clean_food_code)
df_1 = pd.read_excel(file_path[0])
df_2 = pd.read_excel(file_path[1])
df_3 = pd.read_excel(file_path[2])

clean_food_code(df_1)
clean_food_code(df_2)
clean_food_code(df_3)


'餐次  序号 食物名称 主要成分     食物编码  可食部 (克/份)  价格 (元/份)  是否可半份    水分  能量 (kcal)  \\\n0    早餐   1   牛奶   牛奶  101101x        200       1.5      0  87.6         65   \n1    早餐   2   酸奶   酸奶   103001        125       1.5      0  85.5         70   \n2    早餐   3   豆浆   黄豆    31101         10       1.5      0  10.2        390   \n3    早餐   4  大米粥   稻米  012001x         15       0.5      0  13.3        346   \n4    早餐   5  小米粥   小米    15101         15       0.5      0  11.6        361   \n..   ..  ..  ...  ...      ...        ...       ...    ...   ...        ...   \n399  晚餐  45   西瓜   西瓜  066201x        100       1.0      0  92.3         31   \n400  晚餐  46   香蕉   香蕉    65033        100       1.0      0  75.8         93   \n401  晚餐  47   柚子   柚子    64301        100       1.0      0  89.0         42   \n402  晚餐  48   苹果   苹果  061101x        100       1.0      0  86.1         53   \n403  晚餐  49   葡萄   葡萄  063101x        100       1.0      0  88.5         45   \n\n     ...  色氨酸   缬氨酸   精氨酸  组氨酸   丙氨酸  天冬氨

In [36]:
import re
def get_column_base_name(column_name):
    """
    获取列名中括号 '(' 前面的字符串。
    如果列名不包含 '(', 则返回整个列名。
    同时去除前后可能的空白字符。
    """
    match = re.match(r'([^（\(]*)', column_name) # 匹配直到第一个中文或英文左括号
    if match:
        return match.group(1).strip() # 返回匹配到的部分并去除前后空白
    return column_name.strip() # 如果没有括号，返回原列名并去除前后空白

columns_df2 = df_2.columns.tolist()
columns_df3 = df_3.columns.to_list()
base_columns_df2 = [get_column_base_name(col) for col in columns_df2]
base_columns_df3 = [get_column_base_name(col) for col in columns_df3]

print("\ndf_2 (附件2) 的列名:", base_columns_df2)
print("df_3 (附件3) 的列名:", base_columns_df3)

# 对比处理后的列名差异
print("\ndf_2 和 df_3 处理后列名的差异:")
set_base_df2 = set(base_columns_df2)
set_base_df3 = set(base_columns_df3)

print("df_2 有 但 df_3 没有的列名:", list(set_base_df2 - set_base_df3))
print("df_3 有 但 df_2 没有的列名:", list(set_base_df3 - set_base_df2))
print("df_2 和 df_3 共同的列名:", list(set_base_df2.intersection(set_base_df3)))


df_2 (附件2) 的列名: ['餐次', '食物名称', '主要成分', '食物编码', '可食部', '食用份数', '碳水化合物', '蛋白质', '脂肪', '钙', '铁', '锌', '维生素A', '维生素B1', '维生素B2', '维生素C', '异亮氨酸', '亮氨酸', '赖氨酸', '含硫氨基酸', '芳香族氨基酸', '苏氨酸', '色氨酸', '缬氨酸']
df_3 (附件3) 的列名: ['餐次', '序号', '食物名称', '主要成分', '食物编码', '可食部', '价格', '是否可半份', '水分', '能量', '能量', '蛋白质', '脂肪', '碳水化合物', '不溶性膳食纤维', '胆固醇', '灰分', '总维生素A', '胡萝卜素', '视黄醇', '硫胺素', '核黄素', '烟酸', '维生素C', '钙', '磷', '钾', '钠', '镁', '铁', '锌', '硒', '铜', '锰', '蛋白质.1', '异亮氨酸', '亮氨酸', '赖氨酸', '含硫氨基酸', '含硫氨基酸', '含硫氨基酸', '芳香族氨基酸', '芳香族氨基酸', '芳香族氨基酸', '苏氨酸', '色氨酸', '缬氨酸', '精氨酸', '组氨酸', '丙氨酸', '天冬氨酸', '谷氨酸', '甘氨酸', '脯氨酸', '丝氨酸']

df_2 和 df_3 处理后列名的差异:
df_2 有 但 df_3 没有的列名: ['食用份数', '维生素B2', '维生素A', '维生素B1']
df_3 有 但 df_2 没有的列名: ['价格', '硒', '胆固醇', '谷氨酸', '精氨酸', '钾', '灰分', '蛋白质.1', '不溶性膳食纤维', '核黄素', '脯氨酸', '镁', '锰', '铜', '视黄醇', '总维生素A', '组氨酸', '序号', '能量', '丝氨酸', '烟酸', '丙氨酸', '天冬氨酸', '是否可半份', '水分', '磷', '钠', '硫胺素', '甘氨酸', '胡萝卜素']
df_2 和 df_3 共同的列名: ['含硫氨基酸', '维生素C', '缬氨酸', '碳水化合物', '主要成分', '赖氨酸', '脂肪', '蛋白质', '异亮氨酸', '铁', 

df_3 需要删除的列

'硒', '胆固醇', '谷氨酸', '精氨酸', '钾', '灰分', '蛋白质.1', '不溶性膳食纤维', '核黄素', '脯氨酸', '镁', '锰', '铜', '视黄醇', '总维生素A', '组氨酸', '序号', '能量', '丝氨酸', '烟酸', '丙氨酸', '天冬氨酸', '是否可半份', '水分', '磷', '钠', '硫胺素', '甘氨酸', '胡萝卜素'

df_3 需要改名的列
'总维生素A' -> '维生素A'

In [37]:
def get_column_base_name(column_name):
    """
    获取列名中括号 '(' 或 '（' 前面的字符串。
    如果列名不包含括号，则返回整个列名。
    同时去除前后可能的空白字符。
    """
    # 使用非贪婪匹配，匹配到第一个左右括号中的一个就停止
    match = re.match(r"([^（\(]*)", str(column_name))  # 确保输入是字符串
    if match:
        return match.group(1).strip()
    return str(column_name).strip()  # 如果没有括号，返回原列名并去除前后空白


base_names_to_drop = [
    "硒",
    "胆固醇",
    "谷氨酸",
    "精氨酸",
    "钾",
    "灰分",
    "蛋白质.1",
    "不溶性膳食纤维",
    "核黄素",
    "脯氨酸",
    "镁",
    "锰",
    "铜",
    "视黄醇",
    "组氨酸",
    "序号",
    "丝氨酸",
    "烟酸",
    "丙氨酸",
    "天冬氨酸",
    "水分",
    "磷",
    "钠",
    "硫胺素",
    "甘氨酸",
    "胡萝卜素",
]
base_names_to_rename = {"总维生素A": "维生素A"}
# 找到需要删除的列的完整名称
full_names_to_drop = []
current_columns = df_3.columns.tolist()

print("正在匹配需要删除的列...")
for col in current_columns:
    base_name = get_column_base_name(col)
    if base_name in base_names_to_drop:
        full_names_to_drop.append(col)
        print(f"  匹配到要删除的列: '{col}' (基础名: '{base_name}')")

# 执行删除操作
if full_names_to_drop:
    try:
        # 使用一个新的 DataFrame 来避免 SettingWithCopyWarning
        df_3 = df_3.drop(columns=full_names_to_drop)
        print("\n已删除指定的列。")
    except KeyError as e:
        print(
            f"\n警告: 尝试删除的列 {e} 不存在。请检查基础列名列表或 df_3 的实际列名。"
        )
else:
    print("\n没有匹配到需要删除的列。请检查基础列名列表是否正确。")

df_3.rename(columns=base_names_to_rename)


# 打印处理后的 df_3 的列名，确认修改结果
print("\n处理后 df_3 的列名:", df_3.columns.tolist())

正在匹配需要删除的列...
  匹配到要删除的列: '序号' (基础名: '序号')
  匹配到要删除的列: '水分' (基础名: '水分')
  匹配到要删除的列: '不溶性膳食纤维' (基础名: '不溶性膳食纤维')
  匹配到要删除的列: '胆固醇' (基础名: '胆固醇')
  匹配到要删除的列: '灰分' (基础名: '灰分')
  匹配到要删除的列: '胡萝卜素' (基础名: '胡萝卜素')
  匹配到要删除的列: '视黄醇' (基础名: '视黄醇')
  匹配到要删除的列: '硫胺素' (基础名: '硫胺素')
  匹配到要删除的列: '核黄素' (基础名: '核黄素')
  匹配到要删除的列: '烟酸' (基础名: '烟酸')
  匹配到要删除的列: '磷' (基础名: '磷')
  匹配到要删除的列: '钾' (基础名: '钾')
  匹配到要删除的列: '钠' (基础名: '钠')
  匹配到要删除的列: '镁' (基础名: '镁')
  匹配到要删除的列: '硒' (基础名: '硒')
  匹配到要删除的列: '铜' (基础名: '铜')
  匹配到要删除的列: '锰' (基础名: '锰')
  匹配到要删除的列: '蛋白质.1' (基础名: '蛋白质.1')
  匹配到要删除的列: '精氨酸' (基础名: '精氨酸')
  匹配到要删除的列: '组氨酸' (基础名: '组氨酸')
  匹配到要删除的列: '丙氨酸' (基础名: '丙氨酸')
  匹配到要删除的列: '天冬氨酸' (基础名: '天冬氨酸')
  匹配到要删除的列: '谷氨酸' (基础名: '谷氨酸')
  匹配到要删除的列: '甘氨酸' (基础名: '甘氨酸')
  匹配到要删除的列: '脯氨酸' (基础名: '脯氨酸')
  匹配到要删除的列: '丝氨酸' (基础名: '丝氨酸')

已删除指定的列。

处理后 df_3 的列名: ['餐次', '食物名称', '主要成分', '食物编码', '可食部 (克/份)', '价格 (元/份)', '是否可半份', '能量 (kcal)', '能量 (kJ)', '蛋白质', '脂肪', '碳水化合物', '总维生素A', '维生素C', '钙', '铁', '锌', '异亮氨酸', '亮氨酸', '赖氨酸', '含硫氨基酸(SAA

In [38]:
import re
def get_column_base_name(column_name):
    """
    获取列名中括号 '(' 前面的字符串。
    如果列名不包含 '(', 则返回整个列名。
    同时去除前后可能的空白字符。
    """
    match = re.match(r'([^（\(]*)', column_name) # 匹配直到第一个中文或英文左括号
    if match:
        return match.group(1).strip() # 返回匹配到的部分并去除前后空白
    return column_name.strip() # 如果没有括号，返回原列名并去除前后空白

columns_df2 = df_2.columns.tolist()
columns_df3 = df_3.columns.to_list()
base_columns_df2 = [get_column_base_name(col) for col in columns_df2]
base_columns_df3 = [get_column_base_name(col) for col in columns_df3]

print("\ndf_2 (附件2) 的列名:", base_columns_df2)
print("df_3 (附件3) 的列名:", base_columns_df3)

# 对比处理后的列名差异
print("\ndf_2 和 df_3 处理后列名的差异:")
set_base_df2 = set(base_columns_df2)
set_base_df3 = set(base_columns_df3)

print("df_2 有 但 df_3 没有的列名:", list(set_base_df2 - set_base_df3))
print("df_3 有 但 df_2 没有的列名:", list(set_base_df3 - set_base_df2))
print("df_2 和 df_3 共同的列名:", list(set_base_df2.intersection(set_base_df3)))


df_2 (附件2) 的列名: ['餐次', '食物名称', '主要成分', '食物编码', '可食部', '食用份数', '碳水化合物', '蛋白质', '脂肪', '钙', '铁', '锌', '维生素A', '维生素B1', '维生素B2', '维生素C', '异亮氨酸', '亮氨酸', '赖氨酸', '含硫氨基酸', '芳香族氨基酸', '苏氨酸', '色氨酸', '缬氨酸']
df_3 (附件3) 的列名: ['餐次', '食物名称', '主要成分', '食物编码', '可食部', '价格', '是否可半份', '能量', '能量', '蛋白质', '脂肪', '碳水化合物', '总维生素A', '维生素C', '钙', '铁', '锌', '异亮氨酸', '亮氨酸', '赖氨酸', '含硫氨基酸', '含硫氨基酸', '含硫氨基酸', '芳香族氨基酸', '芳香族氨基酸', '芳香族氨基酸', '苏氨酸', '色氨酸', '缬氨酸']

df_2 和 df_3 处理后列名的差异:
df_2 有 但 df_3 没有的列名: ['食用份数', '维生素B2', '维生素A', '维生素B1']
df_3 有 但 df_2 没有的列名: ['是否可半份', '能量', '价格', '总维生素A']
df_2 和 df_3 共同的列名: ['含硫氨基酸', '维生素C', '缬氨酸', '碳水化合物', '主要成分', '赖氨酸', '脂肪', '蛋白质', '异亮氨酸', '铁', '苏氨酸', '钙', '芳香族氨基酸', '锌', '色氨酸', '食物名称', '食物编码', '可食部', '餐次', '亮氨酸']


In [39]:
df_1.to_excel(file_path[0])
df_2.to_excel(file_path[1])
df_3.to_excel(file_path[2])